## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [1]:
import scipy.signal as scs

In [2]:
import glob

import numpy as np
import scipy as sp
import scipy.io


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

In [3]:
def RunPulseRateAlgorithm(data_fl, ref_fl, mode = 'standard_output'):
    
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    #set sampling frequency to 125 Hz
    fs=125
    
    # Convert the frequency from Beat Per Minute to Hz
    # 40 BPM and 240 BPM is the recommended lower limit and higher limit for heartbeat signal, respectively.
    lower_freq_limit,higher_freq_limit =(40/60,
                                          240/60)
    freq_limits = lower_freq_limit,higher_freq_limit
    
    ### use bandpass filter (BPF) to filter out the signal in order to focus on the typical heartbeat frequency range
    ppg=BPF(ppg, lower_freq_limit,higher_freq_limit, sampling_freq = fs)
    accx=BPF(accx, lower_freq_limit,higher_freq_limit, sampling_freq = fs)
    accy=BPF(accy, lower_freq_limit,higher_freq_limit, sampling_freq = fs)
    accz=BPF(accz, lower_freq_limit,higher_freq_limit, sampling_freq = fs)

    # Compute pulse rate estimates and estimation confidence.
    #########################################################    
    pr, confidence = peakdetection(ppg, accx, accy, accz)  
    #########################################################  
    

    # Load the pulse rate reference/ground truths array
    pr_reference=sp.io.loadmat(ref_fl)['BPM0'].reshape((1,-1))[0] 
    
    # Calculate the error, with considering the difference between reference vs produced pulse rate array length 
    if len(pr_reference) < len(pr):
        errors=abs(pr[:len(pr_reference)]-pr_reference)
    elif len(pr_reference) > len(pr):
        errors=abs(pr_reference[:len(pr)]-pr)
    else :
        errors=abs(pr[:len(pr_reference)]-pr_reference)
        
    confidence=np.array(confidence[:len(pr)]) 

    # standard_output will return errors and confidence only, per project instruction (default)
    if mode == 'standard_output':
        final_output =  errors, confidence
    
    # full_output will return errors, confidence, and calculated pulse rate array. But this output is optional.
    elif mode == 'full_output':
        final_output =  errors, confidence, pr
        print(pr[:100])
    else:
        print('ERROR : Optional input is not recognized')
        final_output = 'ERROR : Optional input is not recognized'
        
    return  final_output

In [4]:
def BPF(signal, lower_freq_limit, higher_freq_limit, sampling_freq):
    
    b, a = scs.butter(3, (lower_freq_limit,higher_freq_limit ), btype='bandpass', fs=sampling_freq)
    filtered_signal=  scs.filtfilt(b, a, signal)
    
    return filtered_signal


In [1]:
wl = 1000 # window length
fs = 125 # sampling frequency

def peakdetection(ppg_st1, accx_st1, accy_st1, accz_st1):
    
    # pr is the produced estimation of pulse rate
    pr=[]
    

    confidence=[]   
    

    
    window_length=wl
    
    # Considering Nyquist theorem, windows shift is set to twice the sampling frequency  
    window_shift=2*fs 
    
    # Sampling the time-domain signal for each window shift length
    for i in range(0, len(ppg_st1) - window_length, window_shift):
        ppg_sample=ppg_st1[i: i + window_length]
        accx_sample=accx_st1[i: i + window_length]
        accy_sample=accy_st1[i: i + window_length]
        accz_sample=accz_st1[i: i + window_length]
        
        # Set the frequency range to be used in the fourier domain
        freqs = np.fft.rfftfreq(window_length, 1/fs)
        # Convert the PPG signal to the frequency domain using 1D discrete Fourier Transform (with real input)
        p_fourier = abs(np.fft.rfft(ppg_sample))
        
        # Convert the 3 accelerometer signals to the frequency domain using 1D discrete Fourier Transform (with real input)
        x_fourier = abs(np.fft.rfft(accx_sample))
        y_fourier = abs(np.fft.rfft(accy_sample))
        z_fourier = abs(np.fft.rfft(accz_sample))
        
        # Accumulate the 3 accelerometer signals into a combined accelerometer signal
        xyz_fourier=np.square(x_fourier)+np.square(y_fourier)+np.square(z_fourier)
        xyz_fourier=np.sqrt(xyz_fourier)

        # Detect any peaks in the PPG signal,
        p_fourier_peaks_xdim,p_fourier_peaks_ydim = sp.signal.find_peaks(p_fourier,height=400)
        # Define the x-coordinate of these peaks (if any),
        p_fourier_peaks_xdim=freqs[p_fourier_peaks_xdim]
        # Define the y-coordinate/height of these peaks (if any),
        p_fourier_peaks_ydim=p_fourier_peaks_ydim['peak_heights']

        # remove any unwanted peaks (which is present outside the heartbeat frequency range) in PPG signal
        p_fourier_freq_range=(p_fourier_peaks_xdim>=40/60)&(p_fourier_peaks_xdim<=240/60)
        p_fourier_peaks_xdim_filtered=p_fourier_peaks_xdim[p_fourier_freq_range]
        p_fourier_peaks_ydim_filtered=p_fourier_peaks_ydim[p_fourier_freq_range]   
        
        # Detect any peaks in the Combined Accelerometer(CA) signal
        xyz_fourier_peaks_xdim,xyz_fourier_peaks_ydim = sp.signal.find_peaks(xyz_fourier,height=5)
        # Define the x-coordinate of these peaks (if any),
        xyz_fourier_peaks_xdim=freqs[xyz_fourier_peaks_xdim]
        # Define the y-coordinate/height of these peaks (if any),
        xyz_fourier_peaks_ydim=xyz_fourier_peaks_ydim['peak_heights']

        # remove any unwanted peaks (which is present outside the heartbeat frequency range) in CA signal
        xyz_fourier_freq_range=(xyz_fourier_peaks_xdim>=40/60)&(xyz_fourier_peaks_xdim<=240/60)
        xyz_fourier_peaks_xdim_filtered=xyz_fourier_peaks_xdim[xyz_fourier_freq_range]
        xyz_fourier_peaks_ydim_filtered=xyz_fourier_peaks_ydim[xyz_fourier_freq_range]

        # When a peak within PPG frequencies is detected, compare the top 4 peaks with top 4 peaks from CA frequencies,
        # and remove any shared peaks in order to avoid artifact due to subject movement
        if len(p_fourier_peaks_xdim_filtered)>0:
            
            # Sort the coordinates based on the highest peaks
            p_fourier_peaks_xdim_filtered=p_fourier_peaks_xdim_filtered[np.argsort(-p_fourier_peaks_ydim_filtered)]
            p_fourier_peaks_ydim_filtered=p_fourier_peaks_ydim_filtered[np.argsort(-p_fourier_peaks_ydim_filtered)] 
            
            # If more than 4 peaks are detected, only process the highest 4 peaks
            if len(p_fourier_peaks_xdim_filtered)>4:
                p_fourier_peaks_xdim_filtered=p_fourier_peaks_xdim_filtered[:4]
                p_fourier_peaks_ydim_filtered=p_fourier_peaks_ydim_filtered[:4]
            
            # If one or more peaks within CA frequencies is detected,....
            if len(xyz_fourier_peaks_xdim_filtered)>0: 
                xyz_fourier_peaks_xdim_filtered=xyz_fourier_peaks_xdim_filtered[np.argsort(-xyz_fourier_peaks_ydim_filtered)]
                xyz_fourier_peaks_ydim_filtered=xyz_fourier_peaks_ydim_filtered[np.argsort(-xyz_fourier_peaks_ydim_filtered)]
            
            # .... process only the highest 4 peaks
            if len(xyz_fourier_peaks_xdim_filtered)>4: 
                xyz_fourier_peaks_xdim_filtered=xyz_fourier_peaks_xdim_filtered[:4]
                xyz_fourier_peaks_ydim_filtered=xyz_fourier_peaks_ydim_filtered[:4]
            
            # Check for unique frequencies (detected frequency peaks in PPG signal that does not exist in CA signal)
            unique_f=[x_coord for x_coord in p_fourier_peaks_xdim_filtered if not x_coord  in xyz_fourier_peaks_xdim_filtered]
            
            # Choose only the first detected unique frequency (if there is any) OR simply choose 
            # the first shared peak (PPG-AC) (if unique PPG frequency does not exist) as the pulse rate in Hz
            if len(unique_f)>0:
                pr_f=unique_f[0]
            else:
                pr_f=p_fourier_peaks_xdim_filtered[0]

        # If there is no PPG signal peak detected in the first place, use the strongest frequency in PPG signal as the pulse rate in Hz
        else:
            freqs_range_nopeak=(freqs>=40/60)&(freqs<=240/60)
            freqs_filtered_nopeak=freqs[freqs_range_nopeak]
            p_fourier_filtered_nopeak=p_fourier[freqs_range_nopeak]
            pr_f=freqs_filtered[np.argsort(-p_fourier_filtered)][0]
        
        
        # Convert the pulse rate in Hz to BPM
        pr_bpm=pr_f*60
        
        # Since pulse rate reading can be very fluctuative and unstable, some sort stabilizing algorithm is introduced
        # by updating pulse rate if the calculated pulse rate value is different enough (more than 10 percent difference) 
        # compared to the previous pulse rate reading. ALSO, the updated value is set to ONLY either 10 percent 
        # lower or higher depending on the calculated pulse rate vs recorded pulse rate 
        # Credit to github.com/neilzhang
        if len(pr)>0:
            if pr_bpm>1.1*pr[-1]:
                pr_bpm=1.1*pr[-1]
            if pr_bpm<pr[-1]/1.1:
                pr_bpm=pr[-1]/1.1
                
        # Adding a value to the pulse rate array, it is either a same value with the last reading, 
        # 10 percent higher than the last reading, or 10 percent lower than last reading.
        pr.append(pr_bpm)
        
        # The confidence is quantified as the proportion between highlighted PPG frequency 
        # (which is having slightest difference with detected pulse rate frequency magnitude),
        # compared against total magnitude of frequencies detected
        diff_p_fourier_peaks_xdim=abs(p_fourier_peaks_xdim-pr_bpm/60)
        confidence.append(p_fourier_peaks_ydim[diff_p_fourier_peaks_xdim.argmin()]/sum(p_fourier_peaks_ydim)) 
        
    return pr, confidence

In [32]:
Evaluate()

8.9707247890142803

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

### **Code Description** 
The main processing code to extract the pulse rate value is ` RunPulseRateAlgorithm() ` which is integrated with two secondary processing functions : `BPF()` and `peakdetection()` to execute bandpass filtering and hearbeat signal peak detection, respectively. The default sampling rate is 125 Hz. In-depth details regarding the algorithm are presented with the code as code comments.

In this project, simply run the `Evaluate()` function to execute the whole PPG-Accelerometer processing to extract the pulse rate(s) and subsequently, its error and scoring (aggregate mean error).


### **Data Description** 
The data that is utilized for algorithm development and code validation in this part of the project is Troika dataset. The data consist of Photopletysmograph (PPG) signal reading and 3-axis accelerometer signal reading. The accelerometers and PPG reading are produced simultaneously from record from 12 subjects with age ranging from 18 to 35 years old. Each subject produces 5 minutes of record, while performing different running speeds including rests. The reference heart rate is produced from ECG signal, which is not part of our input. Improvements on the data can be done by adding wider range of subject, longer recording duration, and/or installation of extra PPG sensors.


### **Algorithm Description** 

The algorithm receives input of PPG signal reading and 3-axis accelerometer signal reading (X-axis, Y-Axis, and Z-Axis). It returns pulse rate array and confidence array.  The algorithm works by bandpass-filtering the signals (PPG and Accelerometer components) as its first step. Then, each signal is divided into parts by a same length, which is based on the window length. The window length is chosen carefully so that it would sample the overall signal ideally so that the sample signal is long enough for pulse rate detection and short enough to produce actual and updated result. All signals are converted into fourier domain via 1D Fourier Transform. The three accelerometers signals are accumulated because we will focus on the frequencies.

Some works are to be done (See code comments for more details) to see which PPG signal frequencies that is strong enough in magnitude and different enough from accelerometer frequency. This distinguishment is performed because higher accelerometer frequencies indicate subject constant movements (walking, running), while certain range PPG frequencies indicate the heartbeat frequencies(40-240 BPM). The strongest and unique PPG frequency will be assigned as a pulse rate. However, certain stabilizing algorithm needs to be performed to avoid fluctuative pulse rate reading. After a pulse rate value is chosen, it is put into a series of pulse rate reading which is in a form of array.

The resulted pulse rate array is then compared with the ground truth pulse rate array. The difference between these two arrays, formulated as an error array, will be the first output of the algorithm. 

The confidence is quantified as the proportion between highlighted PPG frequency (which is having slightest difference with the chosen pulse rate frequency magnitude), compared against total magnitude of frequencies detected. This shows how strong a highlighted frequency, which is used as a pulse rate reference, against other (noise) frequencies. The confidence is accumulated for each window on signal sampling, as an array and will be the second output of the algorithm.


The algorithm may not work properly if : 
> - Unsuitable window length is used (too long or too short).
> - Due to the stabilizer algorithm , sudden drastic change on pulse rate may produce inaccurate result.
> - Physical activity movements that mimick the subject's heart rate frequency.


### **Algorithm Performance** 

An absolute mean value based metrics between estimated pulse rate against ground truth pulse rate is utilized to evaluate the algorithm performance, which showed acceptable result. The algorithm should be good for usage with other datasets, as long as the number of features and sensor installations are the same with the development/training dataset counterparts.

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 